In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from scipy.optimize import leastsq

# Load the data
data = pd.read_excel('data.xlsx')

data





In [ ]:
# Preprocessing
label_encoder = LabelEncoder()
data['Potensi_Banjir'] = label_encoder.fit_transform(data['Potensi_Banjir'])  # Mengubah kategori menjadi numerik

# Fitur (Curah_Hujan, Suhu, Tinggi_Muka_Air) dan Target (Potensi_Banjir)
X = data[['Curah_Hujan', 'Suhu', 'Tinggi_Muka_Air']].values
y = data['Potensi_Banjir'].values

# Normalisasi Fitur
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Scaling Target
y = y / np.max(y)  # Scaling target to be between 0 and 1 for sigmoid activation

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Fungsi aktivasi sigmoid
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# Model Neural Network sederhana dengan 1 lapisan tersembunyi
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        # Inisialisasi bobot secara acak dengan lebih kecil agar tidak terlalu besar
        self.W1 = np.random.randn(input_size, hidden_size) * 0.01
        self.W2 = np.random.randn(hidden_size, output_size) * 0.01

    def forward(self, X):
        # Feedforward
        self.Z1 = np.dot(X, self.W1)
        self.A1 = sigmoid(self.Z1)
        self.Z2 = np.dot(self.A1, self.W2)
        self.A2 = sigmoid(self.Z2)
        return self.A2

    def cost_function(self, X, y):
        # Fungsi biaya (mean squared error)
        y_hat = self.forward(X)
        return 0.5 * np.sum((y_hat - y) ** 2)

    def cost_function_prime(self, X, y):
        # Backpropagation untuk menghitung gradien
        y_hat = self.forward(X)
        
        delta2 = np.multiply(-(y - y_hat), y_hat * (1 - y_hat))
        dJdW2 = np.dot(self.A1.T, delta2)
        
        delta1 = np.dot(delta2, self.W2.T) * (self.A1 * (1 - self.A1))
        dJdW1 = np.dot(X.T, delta1)
        
        return dJdW1, dJdW2

    def flatten_weights(self):
        # Memisahkan bobot menjadi satu vektor
        return np.concatenate((self.W1.ravel(), self.W2.ravel()))

    def unflatten_weights(self, flat_weights):
        # Mengubah vektor kembali menjadi bentuk matriks bobot
        hidden_size = self.W1.shape[1]
        self.W1 = flat_weights[:X_train.shape[1] * hidden_size].reshape(X_train.shape[1], hidden_size)
        self.W2 = flat_weights[X_train.shape[1] * hidden_size:].reshape(hidden_size, 1)

    def train(self, X, y, n_iter=500):
        # Optimasi menggunakan Levenberg-Marquardt (leastsq dari SciPy)
        def error_function(flat_weights, X, y):
            self.unflatten_weights(flat_weights)
            y_hat = self.forward(X)
            return (y_hat - y).ravel()

        initial_weights = self.flatten_weights()
        optimal_weights, success = leastsq(error_function, initial_weights, args=(X, y), maxfev=n_iter)
        self.unflatten_weights(optimal_weights)

# Inisialisasi jaringan syaraf tiruan dengan 10 neuron di hidden layer
nn = NeuralNetwork(input_size=X_train.shape[1], hidden_size=10, output_size=1)

# Latih model menggunakan data latih, tingkatkan jumlah iterasi
nn.train(X_train, y_train.reshape(-1, 1), n_iter=1000)

# Prediksi pada data uji
y_pred = nn.forward(X_test)
y_pred_class = np.round(y_pred * np.max(data['Potensi_Banjir'])).astype(int).ravel()  # Mengembalikan skala ke bentuk asli

# Mapping manual angka ke label
label_mapping = {0: 'Aman', 1: 'Waspada', 2: 'Siaga', 3: 'Awas'}
y_pred_labels = [label_mapping[val] for val in y_pred_class]
y_test_labels = [label_mapping[val] for val in np.round(y_test).astype(int)]

# Evaluasi performa
accuracy = np.mean(np.array(y_pred_labels) == np.array(y_test_labels))
print(f'Akurasi Model: {accuracy * 100:.2f}%')

# Tampilkan hasil prediksi dan nilai asli
comparison = pd.DataFrame({'Asli': y_test_labels, 'Prediksi': y_pred_labels})

# Gabungkan hasil prediksi dengan data uji
data_uji = pd.DataFrame(X_test, columns=['Curah_Hujan', 'Suhu', 'Tinggi_Muka_Air'])
data_uji['Asli'] = y_test_labels
data_uji['Prediksi'] = y_pred_labels

# Simpan hasil prediksi ke dalam file Excel
file_path = "hasil_data_uji.xlsx"
data_uji.to_excel(file_path, index=False)

print(f'Hasil data uji disimpan ke dalam file {file_path}')
print("\nHasil Prediksi vs Aslinya:")

comparison.head(20)

In [ ]:
import plotly.graph_objs as go
import plotly.express as px

# Hitung berapa yang cocok dan tidak cocok
comparison['Cocok'] = comparison['Asli'] == comparison['Prediksi']
cocok_count = comparison['Cocok'].sum()
tidak_cocok_count = len(comparison) - cocok_count

# Data untuk visualisasi
labels = ['Cocok', 'Tidak Cocok']
values = [cocok_count, tidak_cocok_count]


# Grafik pie - Persentase prediksi yang cocok dan tidak cocok
pie_data = go.Figure(go.Pie(labels=labels, values=values, hole=0.4))
pie_data.update_layout(title='Persentase Prediksi yang Cocok dan Tidak Cocok')

# Tampilkan grafik
pie_data.show()


In [ ]:
# Hitung frekuensi prediksi berdasarkan kategori
frekuensi_prediksi = pd.Series(y_pred_labels).value_counts().sort_index()

# Hitung persentase masing-masing kategori dan bulatkan menjadi dua desimal
persentase_prediksi = ((frekuensi_prediksi / len(y_pred_labels)) * 100).round(2)

# Buat dataframe untuk menampilkan jumlah dan persentase
prediksi_summary = pd.DataFrame({
    'Kategori': frekuensi_prediksi.index,
    'Jumlah': frekuensi_prediksi.values,
    'Persentase (%)': persentase_prediksi.values
})

# Tampilkan tabel
print(prediksi_summary)

# Warna untuk setiap kategori
colors = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA']

# Visualisasi frekuensi prediksi dengan Plotly
bar_frekuensi = go.Figure([go.Bar(x=prediksi_summary['Kategori'], 
                                  y=prediksi_summary['Jumlah'], 
                                  text=prediksi_summary['Persentase (%)'], 
                                  textposition='auto', 
                                  marker_color=colors)])

bar_frekuensi.update_layout(title='Frekuensi dan Persentase Hasil Prediksi Berdasarkan Kategori', 
                            xaxis_title='Kategori', 
                            yaxis_title='Jumlah Prediksi')

# Tampilkan grafik batang
bar_frekuensi.show()


In [ ]:
import folium
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Load the data
data = pd.read_excel('data.xlsx')

# Koordinat untuk wilayah unik
coordinates = {
    'Baktiya': [5.0621243, 97.3258354],
    'Lhoksukon': [5.0517222, 97.3078233],
    'Langkahan': [4.9211586, 97.1261701],
    'Cot Girek': [4.8616275, 97.2673567],
    'Matangkuli': [5.0306322, 97.2316173],
    'Tanah Luas': [4.9826373, 97.0425453],
    'Stamet Aceh Utara': [5.228798, 96.9449662]
}

# Pisahkan data menjadi latih dan uji
X = data[['Curah_Hujan', 'Suhu', 'Tinggi_Muka_Air']]
y = data['Potensi_Banjir']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tandai data latih dan uji di dataset
data['Set'] = np.where(data.index.isin(X_train.index), 'Latih', 'Uji')

# Filter data latih
data_latih = data[data['Set'] == 'Latih']

# Mapping status potensi banjir ke warna yang didukung oleh Folium
status_to_color = {
    'Aman': 'lightgreen',
    'Waspada': 'orange',
    'Siaga': 'beige',
    'Awas': 'darkred'
}

# CSS custom untuk popup
css_style = """
<style>
    h4{ font-family: Arial, sans-serif; }
    .popup-table { font-family: Arial, sans-serif; border-collapse: collapse; width: 100%; }
    .popup-table td, .popup-table th { border: 1px solid #ddd; padding: 8px; text-align: left; }
    .popup-table tr:nth-child(even) { background-color: #f2f2f2; }
    .popup-table tr:hover { background-color: #ddd; }
    .popup-table th { padding-top: 12px; padding-bottom: 12px; background-color: #4CAF50; color: white; }
</style>
"""

# Membuat peta untuk data latih
map_latih = folium.Map(location=[5.22, 97.22], zoom_start=10)

# Menambahkan marker untuk setiap wilayah data latih
for wilayah in data_latih['Wilayah'].unique():
    # Subset data untuk setiap wilayah
    wilayah_data = data_latih[data_latih['Wilayah'] == wilayah]
    
    # Menghitung frekuensi dan persentase setiap status
    status_counts = wilayah_data['Potensi_Banjir'].value_counts()
    total_status = status_counts.sum()
    status_percentages = (status_counts / total_status * 100).round(2)
    
    # Menentukan status potensi banjir yang paling sering muncul
    status_terbanyak = status_counts.idxmax()
    
    # Menampilkan tabel dengan jumlah dan persentase dalam bentuk HTML yang bisa di-scroll
    table_html = f"""
    {css_style}
    <table class="popup-table">
        <thead>
            <tr>
                <th>Status</th>
                <th>Jumlah</th>
                <th>Persentase (%)</th>
            </tr>
        </thead>
        <tbody>
    """
    
    # Menambahkan baris untuk setiap status
    for status, jumlah in status_counts.items():
        persentase = status_percentages[status]
        table_html += f"""
        <tr>
            <td>{status}</td>
            <td>{jumlah}</td>
            <td>{persentase}</td>
        </tr>
        """
    
    table_html += "</tbody></table>"
    
    # Menambahkan marker dengan popup dan warna berdasarkan status terbanyak
    iframe = folium.IFrame(html=table_html, width=500, height=300)
    popup = folium.Popup(iframe, max_width=500)
    folium.Marker(location=coordinates[wilayah], popup=popup, icon=folium.Icon(color=status_to_color[status_terbanyak])).add_to(map_latih)

# Tampilkan peta latih
map_latih.save("map_latih.html")
map_latih


In [ ]:
# Filter data uji
data_uji = data[data['Set'] == 'Uji']

# Membuat peta untuk data uji
map_uji = folium.Map(location=[5.22, 97.22], zoom_start=10)

# Menambahkan marker untuk setiap wilayah data uji
for wilayah in data_uji['Wilayah'].unique():
    # Subset data untuk setiap wilayah
    wilayah_data = data_uji[data_uji['Wilayah'] == wilayah]
    
    # Menghitung frekuensi dan persentase setiap status
    status_counts = wilayah_data['Potensi_Banjir'].value_counts()
    total_status = status_counts.sum()
    status_percentages = (status_counts / total_status * 100).round(2)
    
    # Menentukan status potensi banjir yang paling sering muncul
    status_terbanyak = status_counts.idxmax()
    
    # Menampilkan tabel dengan jumlah dan persentase dalam bentuk HTML yang bisa di-scroll
    table_html = f"""
    {css_style}
    <table class="popup-table">
        <thead>
            <tr>
                <th>Status</th>
                <th>Jumlah</th>
                <th>Persentase (%)</th>
            </tr>
        </thead>
        <tbody>
    """
    
    # Menambahkan baris untuk setiap status
    for status, jumlah in status_counts.items():
        persentase = status_percentages[status]
        table_html += f"""
        <tr>
            <td>{status}</td>
            <td>{jumlah}</td>
            <td>{persentase}</td>
        </tr>
        """
    
    table_html += "</tbody></table>"
    
    # Menambahkan marker dengan popup dan warna berdasarkan status terbanyak
    iframe = folium.IFrame(html=table_html, width=500, height=300)
    popup = folium.Popup(iframe, max_width=500)
    folium.Marker(location=coordinates[wilayah], popup=popup, icon=folium.Icon(color=status_to_color[status_terbanyak])).add_to(map_uji)

# Tampilkan peta uji
map_uji.save("map_uji.html")
map_uji


In [ ]:
# Filter data uji
data_uji = data[data['Set'] == 'Uji']

# Membuat peta untuk data uji
map_uji = folium.Map(location=[5.22, 97.22], zoom_start=10)

# Menambahkan marker untuk setiap wilayah data uji
for wilayah in data_uji['Wilayah'].unique():
    # Subset data untuk setiap wilayah
    wilayah_data = data_uji[data_uji['Wilayah'] == wilayah]
    
    # Menghitung frekuensi dan persentase setiap status
    status_counts = wilayah_data['Potensi_Banjir'].value_counts()
    total_status = status_counts.sum()
    status_percentages = (status_counts / total_status * 100).round(2)
    
    # Mendapatkan prediksi terbaru (paling baru)
    prediksi_terbaru = wilayah_data.sort_values(by='Tahun', ascending=False).iloc[0]['Potensi_Banjir']
    
    # Koordinat wilayah
    latitude, longitude = coordinates[wilayah]
    
    # Tabel Keterangan Wilayah dan Status Terkini
    table_keterangan_html = f"""
    {css_style}

    <h4>Hasil Prediksi</h4>
    <table class="popup-table">
        <thead>
            <tr>
                <th>Keterangan</th>
                <th>Nilai</th>
            </tr>
        </thead>
        <tbody>
            <tr>
                <td>Nama Wilayah</td>
                <td>{wilayah}</td>
            </tr>
            <tr>
                <td>Koordinat</td>
                <td>{latitude}, {longitude}</td>
            </tr>
            <tr>
                <td>Status Terkini</td>
                <td>{prediksi_terbaru}</td>
            </tr>
        </tbody>
    </table>
    """
    
    # Tabel Historis
    table_historis_html = f"""
    {css_style}
    
    <h4>Historis Banjir</h4>
    <table class="popup-table">
        <thead>
            <tr>
                <th>Bulan</th>
                <th>Tahun</th>
                <th>Status Potensi Banjir</th>
            </tr>
        </thead>
        <tbody>
    """
    
    # Menambahkan baris untuk setiap tahun dan status historis
    for _, row in wilayah_data.iterrows():
        table_historis_html += f"""
        <tr>
            <td>{row['Bulan']}</td>
            <td>{row['Tahun']}</td>
            <td>{row['Potensi_Banjir']}</td>
        </tr>
        """
    
    table_historis_html += "</tbody></table>"
    
    # Tabel Persentase Status Historis
    table_persentase_html = f"""
    {css_style}
    <h4>Frekuensi Status Kerawanan</h4>
    <table class="popup-table">
        <thead>
            <tr>
                <th>Status</th>
                <th>Jumlah</th>
                <th>Persentase (%)</th>
            </tr>
        </thead>
        <tbody>
    """
    
    # Menambahkan baris untuk setiap status historis
    for status, jumlah in status_counts.items():
        persentase = status_percentages[status]
        table_persentase_html += f"""
        <tr>
            <td>{status}</td>
            <td>{jumlah}</td>
            <td>{persentase}</td>
        </tr>
        """
    
    table_persentase_html += "</tbody></table>"
    
    # Gabungkan ketiga tabel
    full_popup_html = f"{table_keterangan_html}<br>{table_historis_html}<br>{table_persentase_html}"
    
    # Menambahkan marker dengan popup dan warna berdasarkan prediksi terbaru
    iframe = folium.IFrame(html=full_popup_html, width=500, height=400)
    popup = folium.Popup(iframe, max_width=500)
    
    # Tentukan warna pin berdasarkan prediksi terbaru
    folium.Marker(location=[latitude, longitude], popup=popup, icon=folium.Icon(color=status_to_color[prediksi_terbaru])).add_to(map_uji)

# Tampilkan peta uji
map_uji.save("map_uji.html")
map_uji
